# 1. ETL

In [11]:
import requests
import geopandas as gpd
from pathlib import Path
import numpy as np
import os
import pandas as pd
from shapely.geometry import Point

In [12]:
ROOT = Path(os.path.abspath('')).resolve().parents[0]
DATA = os.path.join(ROOT, "data")
EXTERNAL_DATA = os.path.join(DATA, "external") 
INTERIM_DATA = os.path.join(DATA, "interim")
RAW_DATA = os.path.join(DATA, "raw")


## Load initial dataset

* Full description of dataset is located in [/docs/datensatzbeschreibung.pdf](../docs/datensatzbeschreibung.pdf)
* [Source](https://daten.berlin.de/datensaetze/fahrraddiebstahl-in-berlin)

In [13]:
df = pd.read_csv(
    os.path.join(RAW_DATA, './Bicycle Theft Data.csv'),
    encoding='cp1252'
)

In [14]:
df.columns = df.columns.str.lower()
df.columns

Index(['angelegt_am', 'tatzeit_anfang_datum', 'tatzeit_anfang_stunde',
       'tatzeit_ende_datum', 'tatzeit_ende_stunde', 'lor', 'schadenshoehe',
       'versuch', 'art_des_fahrrads', 'delikt', 'erfassungsgrund'],
      dtype='object')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35729 entries, 0 to 35728
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   angelegt_am            35729 non-null  object
 1   tatzeit_anfang_datum   35729 non-null  object
 2   tatzeit_anfang_stunde  35729 non-null  int64 
 3   tatzeit_ende_datum     35729 non-null  object
 4   tatzeit_ende_stunde    35729 non-null  int64 
 5   lor                    35729 non-null  int64 
 6   schadenshoehe          35729 non-null  int64 
 7   versuch                35729 non-null  object
 8   art_des_fahrrads       35729 non-null  object
 9   delikt                 35729 non-null  object
 10  erfassungsgrund        35729 non-null  object
dtypes: int64(4), object(7)
memory usage: 3.0+ MB


In [16]:
df.describe()

,tatzeit_anfang_stunde,tatzeit_ende_stunde,lor,schadenshoehe
count,35729.000000,35729.000000,3.572900e+04,35729.000000
mean,14.659464,13.566990,5.425391e+06,1263.032159
std,5.348103,5.441687,3.419558e+06,1212.690923
min,0.000000,0.000000,1.100101e+06,0.000000
25%,10.000000,9.000000,2.300419e+06,500.000000
50%,16.000000,14.000000,4.501041e+06,880.000000
75%,19.000000,18.000000,8.100311e+06,1590.000000
max,23.000000,23.000000,1.260124e+07,10000.000000


### Data description

From the source file, this is what explanation for columns exist (transalted to english via ChatGPT):

* `ANGELEGT_AM` → Case creation datetime (UTC+01/CEST as stored)
* `TATZEIT_ANFANG_DATUM` → Offence start date
* `TATZEIT_ANFANG_STUNDE` → Offence start hour
* `TATZEIT_ENDE_DATUM` → Offence end date
* `TATZEIT_ENDE_STUNDE` → Offence end hour
* `LOR` → LOR planning area ID (8-digit PLR code)
* `SCHADENSHOEHE` (SCHADENSHÖHE) → Loss amount (EUR)
* `VERSUCH` → Attempt (yes/no)
* `ART DES FAHRRADS` → Bicycle type
* `DELIKT` → Offence group
* `ERFASSUNGSGRUND` → Recording basis (offence classification)

Rename columns accordingly:

In [17]:
df.columns = [
    'created_at',
    'start_date',
    'start_hour',
    'end_date',
    'end_hour',
    'lor',
    'price',
    'attempt',
    'bicycle_type',
    'group',
    'type'
]

In [18]:
df.dtypes

created_at      object
start_date      object
start_hour       int64
end_date        object
end_hour         int64
lor              int64
price            int64
attempt         object
bicycle_type    object
group           object
type            object
dtype: object

In [19]:
for column in ['created_at', 'start_date', 'end_date']:
    df[column] = pd.to_datetime(df[column], errors='coerce')

/var/folders/v0/8r5h4ym13fb_mzmqc1hfkt580000gn/T/ipykernel_86987/2565069612.py:2: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[column] = pd.to_datetime(df[column], errors='coerce')
/var/folders/v0/8r5h4ym13fb_mzmqc1hfkt580000gn/T/ipykernel_86987/2565069612.py:2: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[column] = pd.to_datetime(df[column], errors='coerce')


In [20]:
df.dtypes

created_at      datetime64[ns]
start_date      datetime64[ns]
start_hour               int64
end_date        datetime64[ns]
end_hour                 int64
lor                      int64
price                    int64
attempt                 object
bicycle_type            object
group                   object
type                    object
dtype: object

In [62]:
df.describe()

,created_at,start_date,start_hour,end_date,end_hour,price,attempt
count,13618,35729,35729.000000,35729,35729.000000,35729.000000,35729.000000
mean,2024-11-28 22:19:13.649581568,2024-11-21 06:15:06.221836800,14.659464,2024-11-21 21:15:57.910940928,13.566990,1263.032159,1.003918
min,2024-01-01 00:00:00,2024-01-01 00:00:00,0.000000,2024-01-01 00:00:00,0.000000,0.000000,0.000000
25%,2024-06-06 00:00:00,2024-06-12 00:00:00,10.000000,2024-06-12 00:00:00,9.000000,500.000000,1.000000
50%,2024-11-07 00:00:00,2024-10-24 00:00:00,16.000000,2024-10-25 00:00:00,14.000000,880.000000,1.000000
75%,2025-06-02 00:00:00,2025-05-14 00:00:00,19.000000,2025-05-14 00:00:00,18.000000,1590.000000,1.000000
max,2025-12-10 00:00:00,2025-11-02 00:00:00,23.000000,2025-11-02 00:00:00,23.000000,10000.000000,2.000000
std,NaN,NaN,5.348103,NaN,5.441687,1212.690923,0.065536


In [21]:
df['bicycle_type'].unique()

array(['Damenfahrrad', 'Herrenfahrrad', 'Mountainbike', 'Fahrrad',
       'diverse Fahrräder', 'Kinderfahrrad', 'Lastenfahrrad', 'Rennrad'],
      dtype=object)

In [22]:
df['attempt'].unique()

array(['Nein', 'Ja', 'Unbekannt'], dtype=object)

In [23]:
df['type'].unique()

array(['Sonstiger schwerer Diebstahl von Fahrrädern',
       'Sonstiger schwerer Diebstahl in/aus Keller/Boden von Fahrrädern',
       'Einfacher Diebstahl von Fahrrädern',
       'Einfacher Diebstahl aus Keller/Boden von Fahrrädern'],
      dtype=object)

In [24]:
df['group'].unique()

array(['Fahrraddiebstahl', 'Keller- und Bodeneinbruch'], dtype=object)

Map column values to english:

In [25]:
bicycle_type_mapping = {
    "Damenfahrrad": "step_through",
    "Herrenfahrrad": "diamond_frame",
    "Mountainbike": "mtb",
    "Fahrrad": "generic",
    "diverse Fahrräder": "multiple",
    "Kinderfahrrad": "kids",
    "Lastenfahrrad": "cargo",
    "Rennrad": "road"
}

group_mapping = {
    "Fahrraddiebstahl": "bicycle_theft",
    "Keller- und Bodeneinbruch": "cellar_attic_burglary"
}

type_mapping = {
    "Sonstiger schwerer Diebstahl von Fahrrädern": "other_aggravated_bicycle_theft",
    "Sonstiger schwerer Diebstahl in/aus Keller/Boden von Fahrrädern": "other_aggravated_bicycle_theft_cellar_attic",
    "Einfacher Diebstahl von Fahrrädern": "simple_bicycle_theft",
    "Einfacher Diebstahl aus Keller/Boden von Fahrrädern": "simple_bicycle_theft_cellar_attic",
}

attempt_mapping = {
    'Unbekannt': 0,
    'Nein': 1,
    'Ja': 2
}

df['bicycle_type'] = df['bicycle_type'].map(bicycle_type_mapping)
df['group'] = df['group'].map(group_mapping)
df['type'] = df['type'].map(type_mapping)
df['attempt'] = df['attempt'].map(attempt_mapping).astype(int)
df["lor"] = (
    pd.to_numeric(df["lor"], errors="coerce")
      .astype("Int64")
      .astype(str)
      .str.replace(r"\.0$", "", regex=True)
      .str.replace(r"\D", "", regex=True)
      .str.zfill(8)
)

In [26]:
df

,created_at,start_date,start_hour,end_date,end_hour,lor,price,attempt,bicycle_type,group,type
0,2025-02-11,2025-10-31,15,2025-10-31,16,07601546,999,1,step_through,bicycle_theft,other_aggravated_bicycle_theft
1,2025-02-11,2025-11-01,12,2025-11-01,18,01200522,1500,1,diamond_frame,bicycle_theft,other_aggravated_bicycle_theft
2,2025-02-11,2025-11-01,0,2025-11-01,0,01300836,100,1,diamond_frame,cellar_attic_burglary,other_aggravated_bicycle_theft_cellar_attic
3,2025-02-11,2025-11-02,14,2025-11-02,16,03601243,240,1,mtb,bicycle_theft,other_aggravated_bicycle_theft
4,2025-02-11,2025-10-22,12,2025-10-22,20,04501153,399,1,diamond_frame,bicycle_theft,other_aggravated_bicycle_theft
...,...,...,...,...,...,...,...,...,...,...,...
35724,2024-02-01,2024-01-01,16,2024-01-01,20,04500938,1189,1,kids,bicycle_theft,other_aggravated_bicycle_theft
35725,2024-02-01,2024-01-01,17,2024-01-02,12,04400727,2900,1,diamond_frame,bicycle_theft,other_aggravated_bicycle_theft
35726,2024-01-01,2024-01-01,14,2024-01-01,16,06300632,899,1,multiple,bicycle_theft,other_aggravated_bicycle_theft
35727,2024-01-01,2024-01-01,19,2024-01-01,19,10300731,1,1,multiple,bicycle_theft,other_aggravated_bicycle_theft


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35729 entries, 0 to 35728
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   created_at    13618 non-null  datetime64[ns]
 1   start_date    35729 non-null  datetime64[ns]
 2   start_hour    35729 non-null  int64         
 3   end_date      35729 non-null  datetime64[ns]
 4   end_hour      35729 non-null  int64         
 5   lor           35729 non-null  object        
 6   price         35729 non-null  int64         
 7   attempt       35729 non-null  int64         
 8   bicycle_type  35729 non-null  object        
 9   group         35729 non-null  object        
 10  type          35729 non-null  object        
dtypes: datetime64[ns](3), int64(4), object(4)
memory usage: 3.0+ MB


### Save dataframe

In [28]:
df.to_parquet(
    os.path.join(INTERIM_DATA, './bicycle_theft_utf8.parquet.gzip'),
    index=False, compression='gzip'
)

## Load LORs map dataset

* [Source](https://daten.odis-berlin.de/de/dataset/lor_planungsgraeume_2021/?utm_source=chatgpt.com)

In [29]:
url = "https://tsb-opendata.s3.eu-central-1.amazonaws.com/lor_planungsgraeume_2021/lor_planungsraeume_2021.geojson"
gdf = gpd.read_file(url)

In [30]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   PLR_ID      542 non-null    object  
 1   PLR_NAME    542 non-null    object  
 2   BEZ         542 non-null    object  
 3   STAND       542 non-null    object  
 4   GROESSE_M2  542 non-null    float64 
 5   geometry    542 non-null    geometry
dtypes: float64(1), geometry(1), object(4)
memory usage: 25.5+ KB


In [31]:
gdf.describe()

,GROESSE_M2
count,5.420000e+02
mean,1.644130e+06
std,2.764954e+06
min,1.336420e+05
25%,5.168896e+05
50%,8.105156e+05
75%,1.667798e+06
max,2.373492e+07


Transform LOR value - add leading zero for compatibility:

In [32]:
gdf["lor"] = (
    gdf["PLR_ID"]
      .astype(str)
      .str.replace(r"\D", "", regex=True)
      .str.zfill(8)
)

In [33]:
missing = (df[["lor"]].drop_duplicates()
           .merge(gdf[["lor"]], on="lor", how="left", indicator=True)
           .query("_merge == 'left_only'")["lor"])
print("Was not found in geolayer:", len(missing))
print(missing.head(20).tolist())

Was not found in geolayer: 0
[]


Map code to name of district:

In [34]:
bez_map = {
    "01": "Mitte",
    "02": "Friedrichshain-Kreuzberg",
    "03": "Pankow",
    "04": "Charlottenburg-Wilmersdorf",
    "05": "Spandau",
    "06": "Steglitz-Zehlendorf",
    "07": "Tempelhof-Schöneberg",
    "08": "Neukölln",
    "09": "Treptow-Köpenick",
    "10": "Marzahn-Hellersdorf",
    "11": "Lichtenberg",
    "12": "Reinickendorf",
}
gdf["bez_name"] = gdf["BEZ"].astype(str).str.zfill(2).map(bez_map)

### Save dataframe

In [35]:
gdf.to_parquet(
    os.path.join(EXTERNAL_DATA, 'geo_data.geoparquet.gzip'),
    compression="gzip"
)

### Join map dataframe with initial dataframe

In [36]:
cols_geom = ["lor", "PLR_NAME", "BEZ", 'bez_name', "geometry"]
df_geo = df.merge(gdf[cols_geom], on="lor", how="left")
df_geo.columns = df_geo.columns.str.lower()
df_geo.head(5)

,created_at,start_date,start_hour,end_date,end_hour,lor,price,attempt,bicycle_type,group,type,plr_name,bez,bez_name,geometry
0,2025-02-11,2025-10-31,15,2025-10-31,16,07601546,999,1,step_through,bicycle_theft,other_aggravated_bicycle_theft,Franziusweg,07,Tempelhof-Schöneberg,"MULTIPOLYGON (((389917.274 5805460.874, 389900..."
1,2025-02-11,2025-11-01,12,2025-11-01,18,01200522,1500,1,diamond_frame,bicycle_theft,other_aggravated_bicycle_theft,Elberfelder Straße,01,Mitte,"MULTIPOLYGON (((387304.093 5820870.943, 387311..."
2,2025-02-11,2025-11-01,0,2025-11-01,0,01300836,100,1,diamond_frame,cellar_attic_burglary,other_aggravated_bicycle_theft_cellar_attic,Humboldthain Nordwest,01,Mitte,"MULTIPOLYGON (((389194.203 5821902.514, 389190..."
3,2025-02-11,2025-11-02,14,2025-11-02,16,03601243,240,1,mtb,bicycle_theft,other_aggravated_bicycle_theft,Rodenbergstraße,03,Pankow,"MULTIPOLYGON (((392843.655 5823122.007, 392841..."
4,2025-02-11,2025-10-22,12,2025-10-22,20,04501153,399,1,diamond_frame,bicycle_theft,other_aggravated_bicycle_theft,Babelsberger Straße,04,Charlottenburg-Wilmersdorf,"MULTIPOLYGON (((387086.809 5816385.329, 387089..."


In [37]:
df_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35729 entries, 0 to 35728
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   created_at    13618 non-null  datetime64[ns]
 1   start_date    35729 non-null  datetime64[ns]
 2   start_hour    35729 non-null  int64         
 3   end_date      35729 non-null  datetime64[ns]
 4   end_hour      35729 non-null  int64         
 5   lor           35729 non-null  object        
 6   price         35729 non-null  int64         
 7   attempt       35729 non-null  int64         
 8   bicycle_type  35729 non-null  object        
 9   group         35729 non-null  object        
 10  type          35729 non-null  object        
 11  plr_name      35729 non-null  object        
 12  bez           35729 non-null  object        
 13  bez_name      35729 non-null  object        
 14  geometry      35729 non-null  geometry      
dtypes: datetime64[ns](3), geometry(1), i

In [38]:
df_geo.describe()

,created_at,start_date,start_hour,end_date,end_hour,price,attempt
count,13618,35729,35729.000000,35729,35729.000000,35729.000000,35729.000000
mean,2024-11-28 22:19:13.649581568,2024-11-21 06:15:06.221836800,14.659464,2024-11-21 21:15:57.910940928,13.566990,1263.032159,1.003918
min,2024-01-01 00:00:00,2024-01-01 00:00:00,0.000000,2024-01-01 00:00:00,0.000000,0.000000,0.000000
25%,2024-06-06 00:00:00,2024-06-12 00:00:00,10.000000,2024-06-12 00:00:00,9.000000,500.000000,1.000000
50%,2024-11-07 00:00:00,2024-10-24 00:00:00,16.000000,2024-10-25 00:00:00,14.000000,880.000000,1.000000
75%,2025-06-02 00:00:00,2025-05-14 00:00:00,19.000000,2025-05-14 00:00:00,18.000000,1590.000000,1.000000
max,2025-12-10 00:00:00,2025-11-02 00:00:00,23.000000,2025-11-02 00:00:00,23.000000,10000.000000,2.000000
std,NaN,NaN,5.348103,NaN,5.441687,1212.690923,0.065536


Number of events per district:

In [39]:
df_geo.groupby('bez_name')['created_at'].count().sort_values(ascending=False)

bez_name
Mitte                         2466
Friedrichshain-Kreuzberg      1786
Pankow                        1628
Charlottenburg-Wilmersdorf    1422
Tempelhof-Schöneberg          1158
Neukölln                      1101
Treptow-Köpenick              1079
Steglitz-Zehlendorf            893
Lichtenberg                    837
Reinickendorf                  529
Marzahn-Hellersdorf            365
Spandau                        354
Name: created_at, dtype: int64

## Load weather data

* For weather-related features, let us use [free weather api](https://open-meteo.com/):

In [40]:
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 52.52,
    "longitude": 13.405,
    "start_date": "2024-01-01",
    "end_date": "2025-11-04",
    "daily": ",".join([
        "temperature_2m_mean","temperature_2m_min","temperature_2m_max",
        "precipitation_sum","wind_speed_10m_max","sunshine_duration"
    ]),
    "timezone": "Europe/Berlin",
}
j = requests.get(url, params=params, timeout=60).json()
open_meteo_df = pd.DataFrame(j["daily"])
if "sunshine_duration" in open_meteo_df:
    open_meteo_df["sunshine_h"] = open_meteo_df["sunshine_duration"] / 3600.0
open_meteo_df.head()

,time,temperature_2m_mean,temperature_2m_min,temperature_2m_max,precipitation_sum,wind_speed_10m_max,sunshine_duration,sunshine_h
0,2024-01-01,5.3,3.5,7.4,1.9,19.7,17859.23,4.960897
1,2024-01-02,4.6,2.5,7.3,8.5,20.2,0.00,0.000000
2,2024-01-03,8.8,7.3,10.6,10.8,27.8,3789.20,1.052556
3,2024-01-04,3.8,-2.2,7.3,2.8,33.1,0.00,0.000000
4,2024-01-05,0.4,-0.6,0.9,5.4,21.3,0.00,0.000000


In [41]:
open_meteo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   time                 674 non-null    object 
 1   temperature_2m_mean  674 non-null    float64
 2   temperature_2m_min   674 non-null    float64
 3   temperature_2m_max   674 non-null    float64
 4   precipitation_sum    674 non-null    float64
 5   wind_speed_10m_max   674 non-null    float64
 6   sunshine_duration    674 non-null    float64
 7   sunshine_h           674 non-null    float64
dtypes: float64(7), object(1)
memory usage: 42.3+ KB


In [42]:
open_meteo_df.describe()

,temperature_2m_mean,temperature_2m_min,temperature_2m_max,precipitation_sum,wind_speed_10m_max,sunshine_duration,sunshine_h
count,674.000000,674.000000,674.000000,674.000000,674.000000,674.000000,674.000000
mean,11.898813,7.917507,15.959050,1.772552,17.935460,28646.723472,7.957423
std,7.325856,6.543510,8.307873,3.615499,6.288651,16912.862828,4.698017
min,-7.800000,-13.200000,-4.000000,0.000000,4.000000,0.000000,0.000000
25%,6.000000,2.700000,9.400000,0.000000,13.225000,13820.347500,3.838985
50%,11.950000,8.450000,16.000000,0.200000,17.550000,31887.575000,8.857660
75%,18.175000,13.400000,22.400000,2.100000,21.800000,42823.102500,11.895306
max,30.000000,21.500000,37.700000,44.800000,42.200000,55602.730000,15.445203


In [43]:
open_meteo_df['time'] = pd.to_datetime(open_meteo_df['time'])
open_meteo_df.rename(columns={'time': 'created_at'}, inplace=True)

In [44]:
open_meteo_df.dtypes

created_at             datetime64[ns]
temperature_2m_mean           float64
temperature_2m_min            float64
temperature_2m_max            float64
precipitation_sum             float64
wind_speed_10m_max            float64
sunshine_duration             float64
sunshine_h                    float64
dtype: object

### Save dataframe

In [45]:
open_meteo_df.to_parquet(
    os.path.join(EXTERNAL_DATA, "open_meteo.parquet.gzip"),
    compression='gzip'
)

### Join weather dataframe with initial dataframe

In [46]:
df_geo = df_geo.join(open_meteo_df.set_index('created_at'), on='created_at')

## Load population data

* Full description of dataset is located in [/docs/EWR Datenpool Nov 2023.pdf](../docs/EWR%20Datenpool%20Nov%202023.pdf)
* [Source](https://daten.berlin.de/datensaetze/einwohnerinnen-und-einwohner-in-berlin-in-lor-planungsraumen-am-31-12-2024?utm_source=chatgpt.com)

In [47]:
population_df = pd.read_csv(
    os.path.join(RAW_DATA, 'population.csv'), sep=';'
)
population_df.head(5)

,ZEIT,RAUMID,BEZ,PGR,BZR,PLR,BEZPGR,E_E,E_EM,E_EW,...,E_E95_110,E_EU1,E_E1U6,E_E6U15,E_E15U18,E_E18U25,E_E25U55,E_E55U65,E_E65U80,E_E80U110
0,202412,1100101,1,10,1,1,110,3580.0,1869.0,1711.0,...,3.0,37.0,128.0,203.0,49.0,296.0,1774.0,376.0,455.0,262.0
1,202412,1100102,1,10,1,2,110,2034.0,1113.0,921.0,...,0.0,18.0,73.0,105.0,27.0,144.0,1101.0,304.0,198.0,64.0
2,202412,1100103,1,10,1,3,110,5790.0,3073.0,2717.0,...,13.0,52.0,235.0,374.0,123.0,422.0,2807.0,679.0,780.0,318.0
3,202412,1100104,1,10,1,4,110,4889.0,2587.0,2302.0,...,9.0,43.0,212.0,355.0,129.0,464.0,2680.0,557.0,371.0,78.0
4,202412,1100205,1,10,2,5,110,2917.0,1545.0,1372.0,...,3.0,15.0,97.0,161.0,49.0,238.0,1556.0,310.0,340.0,151.0


In [48]:
population_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 51 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ZEIT       542 non-null    int64  
 1   RAUMID     542 non-null    int64  
 2   BEZ        542 non-null    int64  
 3   PGR        542 non-null    int64  
 4   BZR        542 non-null    int64  
 5   PLR        542 non-null    int64  
 6   BEZPGR     542 non-null    int64  
 7   E_E        540 non-null    float64
 8   E_EM       540 non-null    float64
 9   E_EW       540 non-null    float64
 10  E_E00_01   540 non-null    float64
 11  E_E01_02   540 non-null    float64
 12  E_E02_03   540 non-null    float64
 13  E_E03_05   540 non-null    float64
 14  E_E05_06   540 non-null    float64
 15  E_E06_07   540 non-null    float64
 16  E_E07_08   540 non-null    float64
 17  E_E08_10   540 non-null    float64
 18  E_E10_12   540 non-null    float64
 19  E_E12_14   540 non-null    float64
 20  E_E14_15  

In [49]:
population_df.describe()

,ZEIT,RAUMID,BEZ,PGR,BZR,PLR,BEZPGR,E_E,E_EM,E_EW,...,E_E95_110,E_EU1,E_E1U6,E_E6U15,E_E15U18,E_E18U25,E_E25U55,E_E55U65,E_E65U80,E_E80U110
count,542.0,5.420000e+02,542.000000,542.000000,542.000000,542.000000,542.000000,540.000000,540.000000,540.000000,...,540.000000,540.000000,540.000000,540.000000,540.000000,540.000000,540.00000,540.000000,540.000000,540.000000
mean,202412.0,6.547167e+06,6.247232,29.926199,6.492620,23.557196,654.649446,7216.935185,3575.277778,3641.657407,...,13.703704,56.218519,330.703704,597.177778,189.677778,526.961111,3195.80000,953.288889,894.788889,472.318519
std,0.0,3.362516e+06,3.374601,15.994283,3.951993,13.883841,336.248421,3091.200125,1537.021354,1568.572790,...,13.609761,28.601031,157.856635,293.394689,95.918003,257.819953,1613.02393,431.524632,466.486269,322.510638
min,202412.0,1.100101e+06,1.000000,10.000000,1.000000,1.000000,110.000000,16.000000,10.000000,6.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.00000,7.000000,0.000000,0.000000
25%,202412.0,3.601451e+06,3.000000,20.000000,3.000000,12.000000,360.000000,4784.500000,2416.250000,2417.500000,...,4.000000,36.000000,213.000000,386.750000,121.000000,338.500000,2035.25000,617.250000,524.750000,223.000000
50%,202412.0,6.300578e+06,6.000000,30.000000,6.000000,23.000000,630.000000,7223.500000,3557.000000,3689.000000,...,10.000000,55.000000,319.000000,565.500000,180.000000,508.500000,3074.50000,955.000000,831.000000,414.500000
75%,202412.0,9.301101e+06,9.000000,40.000000,9.000000,34.000000,930.000000,9451.000000,4679.500000,4748.500000,...,19.000000,75.000000,432.250000,757.500000,241.250000,686.000000,4216.25000,1234.500000,1219.750000,645.500000
max,202412.0,1.260124e+07,12.000000,70.000000,20.000000,60.000000,1260.000000,16901.000000,8761.000000,8411.000000,...,98.000000,165.000000,987.000000,1872.000000,589.000000,1561.000000,10147.00000,2401.000000,2992.000000,2135.000000


In [50]:
population_df = population_df.rename(columns={'RAUMID': 'lor'})
population_df['lor'] = (
    population_df['lor'].astype(str)
      .str.replace(r"\D", "", regex=True)
      .str.zfill(8)
)

Map keys from documentation to required values:

In [51]:
rename_keys = {
    "ZEIT": "population_snapshot_date",
    "BEZ": "bez_code",
    "PLR": "plr_code",
}

rename_E = {
    "E_E": "population_total",
    "E_EM": "population_male",
    "E_EW": "population_female",
    "E_E00_01": 'age_0_1',
    "E_E14_15": "age_14_15",
    "E_E15_18": "age_15_18",
    "E_E18_21": "age_18_21",
    "E_E25_27": "age_25_27",
    "E_E55_60": "age_55_60",
    'E_E60_63': 'age_60_64',
    "E_E80_85": "age_80_85",
}
columns_to_rename = {**rename_keys, **rename_E}
columns = ['lor'] + list(rename_keys.values()) + list(rename_E.values()) 
columns_to_rename
population_df = population_df.rename(columns=columns_to_rename)[columns]
population_df = population_df.fillna(0.0)
population_df.loc[population_df["population_total"] <= 0, "population_total"] = np.nan
by_bez_med = (population_df.groupby("bez_code")["population_total"]
                            .transform(lambda s: s.median(skipna=True)))
population_df["population_total"] = population_df["population_total"].fillna(by_bez_med)
population_df

,lor,population_snapshot_date,bez_code,plr_code,population_total,population_male,population_female,age_0_1,age_14_15,age_15_18,age_18_21,age_25_27,age_55_60,age_60_64,age_80_85
0,01100101,202412,1,1,3580.0,1869.0,1711.0,37.0,24.0,49.0,67.0,170.0,173.0,130.0,151.0
1,01100102,202412,1,2,2034.0,1113.0,921.0,18.0,9.0,27.0,50.0,60.0,175.0,72.0,43.0
2,01100103,202412,1,3,5790.0,3073.0,2717.0,52.0,31.0,123.0,133.0,210.0,333.0,184.0,150.0
3,01100104,202412,1,4,4889.0,2587.0,2302.0,43.0,51.0,129.0,134.0,222.0,262.0,191.0,42.0
4,01100205,202412,1,5,2917.0,1545.0,1372.0,15.0,18.0,49.0,66.0,123.0,143.0,100.0,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,12601032,202412,12,32,6250.0,3121.0,3129.0,63.0,96.0,262.0,284.0,158.0,362.0,204.0,209.0
538,12601133,202412,12,33,11448.0,5550.0,5898.0,111.0,141.0,494.0,434.0,275.0,659.0,392.0,380.0
539,12601134,202412,12,34,16006.0,7699.0,8307.0,165.0,202.0,577.0,600.0,427.0,945.0,549.0,681.0
540,12601235,202412,12,35,10610.0,5215.0,5395.0,106.0,172.0,477.0,470.0,284.0,659.0,326.0,328.0


In [52]:
population_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lor                       542 non-null    object 
 1   population_snapshot_date  542 non-null    int64  
 2   bez_code                  542 non-null    int64  
 3   plr_code                  542 non-null    int64  
 4   population_total          542 non-null    float64
 5   population_male           542 non-null    float64
 6   population_female         542 non-null    float64
 7   age_0_1                   542 non-null    float64
 8   age_14_15                 542 non-null    float64
 9   age_15_18                 542 non-null    float64
 10  age_18_21                 542 non-null    float64
 11  age_25_27                 542 non-null    float64
 12  age_55_60                 542 non-null    float64
 13  age_60_64                 542 non-null    float64
 14  age_80_85 

### Save population dataset

In [53]:
population_df.to_parquet(
    os.path.join(INTERIM_DATA, 'population.parquet.gzip'),
    compression='gzip'
)

### Join population dataset with initial

In [54]:
df_geo = df_geo.join(population_df.set_index('lor'), on='lor')
df_geo

,created_at,start_date,start_hour,end_date,end_hour,lor,price,attempt,bicycle_type,group,...,population_male,population_female,age_0_1,age_14_15,age_15_18,age_18_21,age_25_27,age_55_60,age_60_64,age_80_85
0,2025-02-11,2025-10-31,15,2025-10-31,16,07601546,999,1,step_through,bicycle_theft,...,3419.0,3558.0,34.0,73.0,177.0,218.0,119.0,616.0,368.0,372.0
1,2025-02-11,2025-11-01,12,2025-11-01,18,01200522,1500,1,diamond_frame,bicycle_theft,...,5683.0,5838.0,77.0,76.0,234.0,294.0,385.0,772.0,554.0,305.0
2,2025-02-11,2025-11-01,0,2025-11-01,0,01300836,100,1,diamond_frame,cellar_attic_burglary,...,7874.0,7073.0,113.0,146.0,439.0,531.0,645.0,761.0,448.0,226.0
3,2025-02-11,2025-11-02,14,2025-11-02,16,03601243,240,1,mtb,bicycle_theft,...,3446.0,3544.0,60.0,55.0,127.0,114.0,213.0,462.0,207.0,119.0
4,2025-02-11,2025-10-22,12,2025-10-22,20,04501153,399,1,diamond_frame,bicycle_theft,...,3849.0,3714.0,62.0,56.0,231.0,185.0,245.0,510.0,288.0,217.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35724,2024-02-01,2024-01-01,16,2024-01-01,20,04500938,1189,1,kids,bicycle_theft,...,4024.0,4257.0,57.0,53.0,168.0,177.0,244.0,598.0,315.0,359.0
35725,2024-02-01,2024-01-01,17,2024-01-02,12,04400727,2900,1,diamond_frame,bicycle_theft,...,3875.0,4447.0,63.0,59.0,173.0,192.0,221.0,593.0,389.0,430.0
35726,2024-01-01,2024-01-01,14,2024-01-01,16,06300632,899,1,multiple,bicycle_theft,...,6706.0,6989.0,74.0,173.0,484.0,468.0,249.0,1222.0,678.0,645.0
35727,2024-01-01,2024-01-01,19,2024-01-01,19,10300731,1,1,multiple,bicycle_theft,...,3892.0,4034.0,37.0,64.0,185.0,177.0,211.0,614.0,456.0,426.0


## Load trafic dencity dataset

* For loading of traffic density, let us use [Overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API) from [OpenStreetMap](https://wiki.openstreetmap.org/wiki/Main_Page)

In [55]:
query = """
[out:json][timeout:60];
area["name"="Berlin"]["boundary"="administrative"]->.a;
(
  nwr(area.a)["amenity"="bicycle_parking"];
  nwr(area.a)["shop"="bicycle"];
  nwr(area.a)["railway"="station"];
);
out center tags;
"""
r = requests.post("https://overpass-api.de/api/interpreter", data={"data": query})
r.raise_for_status()
elements = r.json()["elements"]

# Transform data to points (node -> lon/lat; way/rel -> center.lon/lat)
rows = []
for el in elements:
    tags = el.get("tags", {})
    if el["type"] == "node":
        lon, lat = el["lon"], el["lat"]
    else:
        c = el.get("center")
        if not c:
            continue
        lon, lat = c["lon"], c["lat"]
    rows.append({"id": f"{el['type']}/{el['id']}", "lon": lon, "lat": lat, **tags})

df_pois = pd.DataFrame(rows)

# Classification of the given data
def classify(rec):
    if rec.get("amenity") == "bicycle_parking": return "bike_parking"
    if rec.get("shop") == "bicycle":            return "bike_shop"
    if rec.get("railway") == "station":         return "rail_station"
    return "other"

df_pois["kind"] = df_pois.apply(classify, axis=1)

pois = gpd.GeoDataFrame(
    df_pois,
    geometry=[Point(xy) for xy in zip(df_pois["lon"], df_pois["lat"])],
    crs=4326
).to_crs(25833)

pois = pois.drop_duplicates(subset=["id"]).reset_index(drop=True)

In [56]:
if pois.crs is None:
    pois = pois.set_crs(4326)
pois = pois.to_crs(25833)

# Get one polygon per LOR and build dataframe based on this
lor_polys = (df_geo[["lor", "geometry"]]
             .dropna(subset=["geometry"])
             .drop_duplicates(subset=["lor"])
             .copy())
gdf_lor = gpd.GeoDataFrame(lor_polys, geometry="geometry")

if gdf_lor.crs is None:
    minx, miny, maxx, maxy = gdf_lor.total_bounds
    if max(abs(minx), abs(maxx)) <= 180 and max(abs(miny), abs(maxy)) <= 90:
        gdf_lor = gdf_lor.set_crs(4326)
    else:
        gdf_lor = gdf_lor.set_crs(25833)

gdf_lor_25833 = gdf_lor.to_crs(25833)

joined = gpd.sjoin(
    pois,
    gdf_lor_25833[["lor", "geometry"]],
    how="inner",
    predicate="within"
)

area_km2 = (gdf_lor_25833.set_index("lor").area / 1e6).rename("area_km2")
poi_cnt = joined.groupby("lor").size().rename("poi_cnt")

poi_density = (poi_cnt.to_frame()
               .join(area_km2, how="right")
               .fillna({"poi_cnt": 0}))
poi_density["poi_density_km2"] = poi_density["poi_cnt"] / poi_density["area_km2"]
poi_density.head(5)

,poi_cnt,area_km2,poi_density_km2
lor,,,
07601546,6.0,1.773447,3.383241
01200522,128.0,0.639144,200.267843
01300836,100.0,1.440714,69.410022
03601243,71.0,0.329525,215.461750
04501153,65.0,0.585827,110.954225


In [57]:
poi_density.info()

<class 'pandas.core.frame.DataFrame'>
Index: 537 entries, 07601546 to 07601548
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   poi_cnt          537 non-null    float64
 1   area_km2         537 non-null    float64
 2   poi_density_km2  537 non-null    float64
dtypes: float64(3)
memory usage: 32.9+ KB


In [58]:
poi_density.describe()

,poi_cnt,area_km2,poi_density_km2
count,537.000000,537.000000,537.000000
mean,47.618250,1.647915,71.040761
std,54.180026,2.775720,101.921768
min,0.000000,0.133571,0.000000
25%,10.000000,0.515675,6.340687
50%,28.000000,0.807539,31.728954
75%,66.000000,1.667094,95.931318
max,443.000000,23.725255,785.802650


### Save traffic density dataset

In [59]:
poi_density.to_parquet(
    os.path.join(EXTERNAL_DATA, 'traffic_density.parquet.gzip'),
    compression='gzip'
)

### Join traffic density dataset with initial

In [60]:
df_geo = df_geo.join(poi_density, on='lor')

## Save combined data

In [61]:
# Cast to GeoDataFrame so that it is possible to save it in .geoparquet format
gdf = gpd.GeoDataFrame(df_geo, geometry="geometry", crs=gdf.crs or 25833)
gdf.to_parquet(
    os.path.join(INTERIM_DATA, 'df_geo_etl.geoparquet.gzip'),
    compression='gzip'
)